In [124]:
'''
#Sequence to sequence example in Keras (character-level).
This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.
**Summary of the algorithm**
- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    It uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.
**Data download**
[English to French sentence pairs.
](http://www.manythings.org/anki/fra-eng.zip)
[Lots of neat sentence pairs datasets.
](http://www.manythings.org/anki/)
**References**
- [Sequence to Sequence Learning with Neural Networks
   ](https://arxiv.org/abs/1409.3215)
- [Learning Phrase Representations using
    RNN Encoder-Decoder for Statistical Machine Translation
    ](https://arxiv.org/abs/1406.1078)
''';

In [125]:
from __future__ import print_function

In [126]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [153]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 1000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'jerry_q_a.txt'

In [154]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r') as f: #, encoding='utf-8'
    lines = f.read().split('\n')

In [155]:
lines

['Do you know what this is all about? Do you know, why were here? To be out, this is out...and out is one of the single most enjoyable experiences of life. People...did you ever hear people talking about We should go out? This is what theyre talking about...this whole thing, were all out now, no one is home. Not one person here is home, were all out! There are people tryin to find us, they dont know where we are. (on an imaginary phone) Did you ring?, I cant find him. Where did he go? He didnt tell me where he was going. He must have gone out. You wanna go out you get ready, you pick out the clothes, right? You take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...Then youre standing around, whatta you do? You go We gotta be getting back. Once youre out, you wanna get back! You wanna go to sleep, you wanna get up, you wanna go out again tomorrow, right? Where ever you are in life, its my feeling, youve gotta go.\t(pointing at Georges s

In [156]:
print(f"We are only going to train with {num_samples:,.0f} samples")
print(f'However, we have {len(lines) - 1:,.0f} potental french-english pairs to work with')

We are only going to train with 1,000 samples
However, we have 14,785 potental french-english pairs to work with


In [157]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [158]:
target_characters

{'\t',
 '\n',
 ' ',
 '!',
 '"',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '8',
 '9',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [159]:
len(input_texts)

1000

In [160]:
input_texts[9999]

IndexError: list index out of range

In [161]:
target_texts[::-1]

['\tLook at that woman feeding her baby greasy, disgusting, coffee shop corned beef hash. Isnt that child abuse?\n',
 '\tIf they were fanatic about showers, they might.\n',
 '\tYou know, the water pressures terrible in my building. And she loves a good shower.\n',
 '\tSo, is there any way out of this Elaine thing?\n',
 '\tMy censoring system broke down. You know that little guy in your head who watches everything you say? Makes sure you dont make a mistake? He went for a cup of coffee. And in that second ruined my life.\n',
 '\tTwo minutes.\n',
 '\tOh, whos gonna know?\n',
 '\tCome in for two minutes and sit with me.\n',
 '\tBut I need the home bed advantage.\n',
 '\tShes right in my building! Right above me! Every time I come in the building, Im gonna have to sneak around like a cat burglar.\n',
 '\tDid you ever ask an ex-girlfriend to move into your building?\n',
 '\tCause Im an idiot! You may think youre an idiot, but with all due respect, Im a much bigger idiot than you are.\n',
 '

In [162]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])
print(f"There are {num_encoder_tokens} unique encoder tokens")
print(f"There are {num_decoder_tokens} unique decoder tokens")
print(f"The max encoder seq length is {max_encoder_seq_length}")
print(f"The max decoder seq length is {max_decoder_seq_length}")

There are 68 unique encoder tokens
There are 72 unique decoder tokens
The max encoder seq length is 1884
The max decoder seq length is 1886


In [163]:
print(target_characters)

['\t', '\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '8', '9', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [164]:
print('Number of samples:', len(input_texts))
print('Number of targets:', len(target_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 1000
Number of targets: 1000
Number of unique input tokens: 68
Number of unique output tokens: 72
Max sequence length for inputs: 1884
Max sequence length for outputs: 1886


In [165]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [166]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [167]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '"': 4,
 "'": 5,
 '(': 6,
 ')': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 '/': 11,
 '0': 12,
 '1': 13,
 '2': 14,
 '3': 15,
 '4': 16,
 '5': 17,
 '8': 18,
 '9': 19,
 ';': 20,
 '?': 21,
 'A': 22,
 'B': 23,
 'C': 24,
 'D': 25,
 'E': 26,
 'F': 27,
 'G': 28,
 'H': 29,
 'I': 30,
 'J': 31,
 'K': 32,
 'L': 33,
 'M': 34,
 'N': 35,
 'O': 36,
 'P': 37,
 'Q': 38,
 'R': 39,
 'S': 40,
 'T': 41,
 'U': 42,
 'V': 43,
 'W': 44,
 'Y': 45,
 'a': 46,
 'b': 47,
 'c': 48,
 'd': 49,
 'e': 50,
 'f': 51,
 'g': 52,
 'h': 53,
 'i': 54,
 'j': 55,
 'k': 56,
 'l': 57,
 'm': 58,
 'n': 59,
 'o': 60,
 'p': 61,
 'q': 62,
 'r': 63,
 's': 64,
 't': 65,
 'u': 66,
 'v': 67,
 'w': 68,
 'x': 69,
 'y': 70,
 'z': 71}

In [168]:
target_texts[0]

'\t(pointing at Georges shirt) See, to me, that button is in the worst possible spot. The second button literally makes or breaks the shirt, look at it. Its too high! Its in no-mans-land. You look like you live with your mother.\n'

In [169]:
import time
from tqdm import tqdm

In [170]:
for i, (input_text, target_text) in tqdm(enumerate(zip(input_texts, target_texts))):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

1000it [00:00, 4434.51it/s]


In [171]:
len(input_texts)

1000

In [172]:
decoder_input_data.shape

(1000, 1886, 72)

In [173]:
decoder_target_data[0,4]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.], dtype=float32)

In [174]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [175]:
encoder_outputs

<tf.Tensor 'lstm_3/TensorArrayReadV3:0' shape=(?, 256) dtype=float32>

In [179]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [177]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [180]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, 68)     0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, None, 72)     0                                            
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, 256), (None, 332800      input_3[0][0]                    
__________________________________________________________________________________________________
lstm_4 (LSTM)                   [(None, None, 256),  336896      input_4[0][0]                    
                                                                 lstm_3[0][1]                     
          

In [178]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('jerry_s2s.h5')

Train on 800 samples, validate on 200 samples
Epoch 1/100
192/800 [======>.......................] - ETA: 3:29 - loss: 0.1575 - acc: 0.3124

KeyboardInterrupt: 

In [249]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [250]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [251]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        #print(output_tokens)
        #print(output_tokens[0, 0, 2])
        print(states_value[1].shape)
        print(states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [252]:
for seq_index in range(2):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    #print(input_seq)
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

(1, 256)
[array([[ 1.74049690e-01,  1.01413287e-01, -2.18195811e-01,
        -1.88570559e-01,  6.59997165e-01, -3.07717323e-01,
        -2.88076252e-01, -3.93630296e-01,  5.40675879e-01,
         2.79174984e-01, -3.75758588e-01,  1.83972027e-02,
         9.74219963e-02,  1.43724501e-01, -2.05511585e-01,
        -3.83587070e-02,  7.74086826e-03,  1.17249794e-01,
         1.45910800e-01, -7.28498995e-02, -3.80688995e-01,
        -4.28968579e-01,  2.72060275e-01, -1.56482100e-01,
         6.06665611e-01, -5.54875322e-02, -3.69019993e-02,
        -3.65527272e-01, -1.24136962e-01, -2.11800590e-01,
         2.53589332e-01,  3.37730497e-01, -3.33062977e-01,
         8.17978919e-01,  2.76785284e-01,  2.50910968e-01,
         3.58735889e-01,  1.93188176e-01,  9.14227605e-01,
         1.01387035e-02, -7.17083737e-02, -3.30238491e-01,
         1.20196491e-01, -3.07589829e-01,  7.15175986e-01,
        -9.05699730e-02, -1.49270609e-01, -1.46433517e-01,
         5.54081142e-01, -6.40270710e-02, -8.5

In [165]:
encoder_input_data[2].shape

(8, 43)

In [140]:
max_encoder_seq_length

8

In [146]:
def encode_sentence_english(sentence):
    encoder_input_sent = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
    
    for t, char in enumerate(sentence):
        print('time', t)
        print('char', char)
        encoder_input_sent[0, t, input_token_index[char]] = 1
        
    return encoder_input_sent

In [260]:
input_sentence = "Attack!"

input_sentence_oneHot = encode_sentence_english(input_sentence)

decoded_sentence = decode_sequence(input_sentence_oneHot)

print(input_sentence)
print(decoded_sentence)

time 0
char A
time 1
char t
time 2
char t
time 3
char a
time 4
char c
time 5
char k
time 6
char !
(1, 256)
[array([[ 6.67638242e-01,  4.16984618e-01, -6.60817862e-01,
         7.32164681e-01, -7.08946511e-02, -2.96950042e-01,
        -4.30774093e-01, -0.00000000e+00,  5.78012824e-01,
        -0.00000000e+00,  0.00000000e+00,  9.81736124e-01,
         0.00000000e+00,  3.15692299e-03,  1.00755185e-01,
        -4.27727791e-04, -0.00000000e+00, -2.34443247e-02,
         5.63348770e-01,  4.70298499e-01, -8.85704041e-01,
        -4.11666811e-01,  5.36973894e-01, -0.00000000e+00,
         0.00000000e+00, -1.93124323e-03, -0.00000000e+00,
         8.56291652e-01, -7.59783745e-01,  3.83032556e-03,
        -1.57281775e-02,  3.62096012e-01, -0.00000000e+00,
         7.74557292e-01,  8.35497305e-02, -9.72686410e-01,
        -0.00000000e+00,  5.09300888e-01,  9.41330910e-01,
         0.00000000e+00, -1.52162090e-02, -2.32992426e-01,
        -9.98246610e-01,  3.08455020e-01, -6.15420640e-01,
       